<a href="https://colab.research.google.com/github/SJeevitha007/NLP_Assignment/blob/main/script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import spacy
pd.set_option('max_colwidth', None)

In [ ]:
!pip install PYPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.0 MB/s eta 0:00:00


In [ ]:
import PyPDF2

files=['chapter-2.pdf','chapter-3.pdf','chapter-2.pdf']
l=[]
for f in files:
  pdf_file = open(f, 'rb')
  reader = PyPDF2.PdfReader(pdf_file)
  for page_num in range(len(reader.pages)):
      page = reader.pages[page_num]
      page_text = page.extract_text()
      paragraphs = page_text.split('\n\n')
      for paragraph in paragraphs:
          l.append(paragraph)
  pdf_file.close()

In [ ]:
len(l)

47

In [ ]:
df=pd.DataFrame(l,columns=['Paragraph'])
df.head()

In [ ]:
import re
def preprocess(text):
    new_text = re.sub(r'[\n\r\t]', ' ', text)
    new_text = re.sub(r'Fig','',text)
    new_text = re.sub(r' +', ' ', new_text)
    return new_text.strip()

In [ ]:
df['Paragraph']=df['Paragraph'].apply(preprocess)

In [ ]:
df.loc[5]

Paragraph    14 OUR PASTS – III\nAfter the defeat at Plassey, Sirajuddaulah was \nassassinated and Mir Jafar made the nawab. The Company \nwas still unwilling to take over the responsibility of administration. Its prime objective was the expansion of trade. If this could be done without conquest, through the help of local rulers who were willing to grant privileges, then territories need not be taken over directly. \nSoon the Company discovered that this was rather \ndifficult. For even the puppet nawabs were not always as helpful as the Company wanted them to be. After all, they had to maintain a basic appearance of dignity and sovereignty if they wanted respect from their subjects. \nWhat could the Company do? When Mir Jafar \nprotested, the Company deposed him and installed Mir Qasim in his place. When Mir Qasim complained, he in turn was defeated in a battle fought at Buxar (1764), driven out of Bengal, and Mir Jafar was reinstalled. The Nawab had to pay Rs. 500,000 every month but

In [ ]:
df['Paragraph']=df['Paragraph'].apply(lambda x: x[:-80])

In [ ]:
!pip install textwrap3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from textwrap3 import wrap
import textwrap

def wrap(text, width=150):
    return "".join(textwrap.wrap(text, width=width))

In [ ]:
df['Paragraph'] = df['Paragraph'].apply(wrap, width=90)

In [ ]:
nlp=spacy.load("en_core_web_sm")
def preprocesss(text):
  doc=nlp(text)
  lis=[]
  for token in doc:
    if token.is_stop or token.like_email or token.like_url or token.is_punct:
      continue
    else:
      lis.append(token.lemma_)
  return " ".join(lis)

In [ ]:
df['Paragraph']=df['Paragraph'].apply(preprocesss)

In [ ]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4393324 sha256=f314fa383ac781411042ab7c42e66fcbeb0dcfb186a8d3bb6c429554e0a225b2
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import fasttext.util

In [ ]:
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')

'cc.en.300.bin'

In [ ]:
model_path = 'cc.en.300.bin'  # replace with your own model path
ft = fasttext.load_model(model_path)

In [ ]:
def extract_keywords_ft(text, n_keywords=10):
    # Generate FastText embeddings for each word in the document
    word_embeddings = [ft.get_word_vector(word) for word in text.split()]

    # Compute the mean of the embeddings to get the document embedding
    doc_embedding = np.mean(word_embeddings, axis=0)

    # Compute the cosine similarity between each word embedding and the document embedding
    similarities = [np.dot(doc_embedding, word_embedding) / (np.linalg.norm(doc_embedding) * np.linalg.norm(word_embedding)) for word_embedding in word_embeddings]

    # Sort the words by their similarity to the document
    keywords = [word for _, word in sorted(zip(similarities, text.split()), reverse=True)]
    keywords = set(keywords)
    keywords = list(keywords)
    return keywords[:n_keywords]

In [ ]:
keywords = df['Paragraph'].apply(extract_keywords_ft)

In [ ]:
from sklearn.cluster import KMeans
import numpy as np


def summary(text, num_sentences=5):
    # Split text into sentences
    sentences = text.split('.')
    num_sentences = min(num_sentences, len(sentences))
    # Generate sentence embeddings
    embeddings = [ft.get_sentence_vector(s) for s in sentences]
    
    # Cluster similar sentences together
    kmeans = KMeans(n_clusters=num_sentences, random_state=0).fit(embeddings)
    
    # Select most representative sentence from each cluster
    summary = []
    for i in range(num_sentences):
        cluster = np.where(kmeans.labels_ == i)[0]
        representative = cluster[np.argmax([np.linalg.norm(embeddings[j] - kmeans.cluster_centers_[i]) for j in cluster])]
        summary.append(sentences[representative].strip())
    
    return ' '.join(summary)


In [ ]:
! pip install openai

In [ ]:
df['summary']=df["Paragraph"].apply(summary)

In [ ]:
df['summary']

In [ ]:
import random
def get_mca_questions(paragraphs, keywords):
    mca_questions = []
    for summary in paragraphs:
        correct_keywords = random.sample(keywords, 2)
        question = f"Q: {summary}?"
        distractor_keywords = [kw for kw in keywords if kw not in correct_keywords]
        distractor_keywords = random.sample(distractor_keywords, 2)

        for i, distractor in enumerate(distractor_keywords):
            question += f"\n{i+1}. {distractor} is related to {correct_keywords[0]}."\\script.py
        for i, distractor in enumerate(distractor_keywords):
            question += f"\n{i+3}. {distractor} is related to {correct_keywords[1]}."

        question += f"\nCorrect Options: (a) & (b)"

        mca_questions.append(question)
        
    return mca_questions

In [ ]:
paragraphs=df['summary'].to_list()
mcq_questions = [get_mca_questions(p, k) for p, k in zip(paragraphs, keywords)]
for question in mcq_questions:
    print(question)


['Q: F?\n1. chloroplasts is related to photosynthesis.\n2. thylakoid membranes is related to photosynthesis.\n3. chloroplasts is related to light energy.\n4. thylakoid membranes is related to light energy.\nCorrect Options: (a) & (b)', 'Q: r?\n1. photosynthesis is related to chloroplasts.\n2. thylakoid membranes is related to chloroplasts.\n3. photosynthesis is related to light energy.\n4. thylakoid membranes is related to light energy.\nCorrect Options: (a) & (b)', 'Q: o?\n1. light energy is related to chloroplasts.\n2. thylakoid membranes is related to chloroplasts.\n3. light energy is related to photosynthesis.\n4. thylakoid membranes is related to photosynthesis.\nCorrect Options: (a) & (b)', 'Q: m?\n1. thylakoid membranes is related to chloroplasts.\n2. light energy is related to chloroplasts.\n3. thylakoid membranes is related to photosynthesis.\n4. light energy is related to photosynthesis.\nCorrect Options: (a) & (b)', 'Q:  ?\n1. photosynthesis is related to light energy.\n2. t

In [ ]:
mcq_questions.to_csv("Objective1.csv")

AttributeError: ignored